In [0]:
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("Demo").getOrCreate()

This program takes a dataset consisted of 30 users rating 100 movies, with a total of 1500 rating samples. The purpose is to train a recommendation system with the alternating least squares algorithm and test which movies it would recommend to users 9 and 13.

In [0]:
movrdd= spark.sparkContext.textFile("/FileStore/tables/movies.csv", 4)
movrdd.collect()

Out[2]: ['movieId,rating,userId',
 '2,3,0',
 '3,1,0',
 '5,2,0',
 '9,4,0',
 '11,1,0',
 '12,2,0',
 '15,1,0',
 '17,1,0',
 '19,1,0',
 '21,1,0',
 '23,1,0',
 '26,3,0',
 '27,1,0',
 '28,1,0',
 '29,1,0',
 '30,1,0',
 '31,1,0',
 '34,1,0',
 '37,1,0',
 '41,2,0',
 '44,1,0',
 '45,2,0',
 '46,1,0',
 '47,1,0',
 '48,1,0',
 '50,1,0',
 '51,1,0',
 '54,1,0',
 '55,1,0',
 '59,2,0',
 '61,2,0',
 '64,1,0',
 '67,1,0',
 '68,1,0',
 '69,1,0',
 '71,1,0',
 '72,1,0',
 '77,2,0',
 '79,1,0',
 '83,1,0',
 '87,1,0',
 '89,2,0',
 '91,3,0',
 '92,4,0',
 '94,1,0',
 '95,2,0',
 '96,1,0',
 '98,1,0',
 '99,1,0',
 '2,2,1',
 '3,1,1',
 '4,2,1',
 '6,1,1',
 '9,3,1',
 '12,1,1',
 '13,1,1',
 '14,1,1',
 '16,1,1',
 '19,1,1',
 '21,3,1',
 '27,1,1',
 '28,3,1',
 '33,1,1',
 '36,2,1',
 '37,1,1',
 '40,1,1',
 '41,2,1',
 '43,1,1',
 '44,1,1',
 '47,1,1',
 '50,1,1',
 '54,1,1',
 '56,2,1',
 '57,1,1',
 '58,1,1',
 '60,1,1',
 '62,4,1',
 '63,1,1',
 '67,1,1',
 '68,4,1',
 '70,2,1',
 '72,1,1',
 '73,1,1',
 '74,2,1',
 '76,1,1',
 '77,3,1',
 '78,1,1',
 '81,1,1',
 '82,1,1',
 '85,3,1',
 '86,2,1',
 '88,2,1',
 '91,1,1',
 '92,2,1',
 '93,1,1',
 '94,2,1',
 '96,1,1',
 '97,1,1',
 '4,3,2',
 '6,1,2',
 '8,5,2',
 '9,1,2',
 '10,1,2',
 '12,3,2',
 '13,1,2',
 '15,2,2',
 '18,2,2',
 '19,4,2',
 '22,1,2',
 '26,1,2',
 '28,1,2',
 '34,4,2',
 '35,1,2',
 '37,5,2',
 '38,1,2',
 '39,5,2',
 '40,4,2',
 '47,1,2',
 '50,1,2',
 '52,2,2',
 '54,1,2',
 '55,1,2',
 '57,2,2',
 '58,2,2',
 '59,1,2',
 '61,1,2',
 '62,1,2',
 '64,1,2',
 '65,1,2',
 '66,3,2',
 '68,1,2',
 '71,3,2',
 '76,1,2',
 '77,1,2',
 '78,1,2',
 '80,1,2',
 '83,5,2',
 '85,1,2',
 '87,2,2',
 '88,1,2',
 '89,4,2',
 '90,1,2',
 '92,4,2',
 '93,5,2',
 '0,1,3',
 '1,1,3',
 '2,1,3',
 '7,3,3',
 '8,3,3',
 '9,1,3',
 '14,1,3',
 '15,1,3',
 '16,1,3',
 '18,4,3',
 '19,1,3',
 '24,3,3',
 '26,1,3',
 '29,3,3',
 '33,1,3',
 '34,3,3',
 '35,1,3',
 '36,3,3',
 '37,1,3',
 '38,2,3',
 '43,1,3',
 '44,1,3',
 '46,1,3',
 '47,1,3',
 '51,5,3',
 '52,3,3',
 '56,1,3',
 '58,1,3',
 '60,3,3',
 '62,1,3',
 '65,2,3',
 '66,1,3',
 '67,1,3',
 '68,2,3',
 '70,1,3',
 '72,2,3',
 '76,3,3',
 '79,3,3',
 '80,4,3',
 '81,1,3',
 '83,1,3',
 '84,1,3',
 '86,1,3',
 '87,2,3',
 '88,4,3',
 '89,1,3',
 '91,1,3',
 '94,3,3',
 '1,1,4',
 '6,1,4',
 '8,1,4',
 '9,1,4',
 '10,1,4',
 '11,1,4',
 '12,1,4',
 '13,1,4',
 '14,2,4',
 '15,1,4',
 '17,1,4',
 '20,1,4',
 '22,1,4',
 '23,1,4',
 '24,1,4',
 '29,4,4',
 '30,1,4',
 '31,1,4',
 '34,1,4',
 '35,1,4',
 '36,1,4',
 '39,2,4',
 '40,3,4',
 '41,4,4',
 '43,2,4',
 '44,1,4',
 '45,1,4',
 '46,1,4',
 '47,1,4',
 '49,2,4',
 '50,1,4',
 '51,1,4',
 '52,4,4',
 '54,1,4',
 '55,1,4',
 '60,3,4',
 '61,1,4',
 '62,4,4',
 '63,3,4',
 '65,1,4',
 '67,2,4',
 '69,1,4',
 '70,4,4',
 '71,1,4',
 '73,1,4',
 '78,1,4',
 '84,1,4',
 '85,1,4',
 '87,3,4',
 '88,3,4',
 '89,2,4',
 '96,1,4',
 '97,1,4',
 '98,1,4',
 '99,1,4',
 '0,1,5',
 '1,1,5',
 '4,1,5',
 '5,1,5',
 '8,1,5',
 '9,3,5',
 '10,2,5',
 '13,3,5',
 '15,1,5',
 '19,1,5',
 '20,3,5',
 '21,2,5',
 '23,3,5',
 '27,1,5',
 '28,1,5',
 '29,1,5',
 '31,1,5',
 '36,3,5',
 '38,2,5',
 '39,1,5',
 '42,1,5',
 '48,3,5',
 '49,4,5',
 '50,3,5',
 '51,1,5',
 '52,1,5',
 '54,1,5',
 '55,5,5',
 '56,3,5',
 '58,1,5',
 '60,1,5',
 '61,1,5',
 '64,3,5',
 '65,2,5',
 '68,4,5',
 '70,1,5',
 '71,1,5',
 '72,1,5',
 '74,1,5',
 '79,1,5',
 '81,2,5',
 '84,1,5',
 '85,1,5',
 '86,1,5',
 '88,1,5',
 '90,4,5',
 '91,2,5',
 '95,2,5',
 '99,1,5',
 '0,1,6',
 '1,1,6',
 '2,3,6',
 '5,1,6',
 '6,1,6',
 '9,1,6',
 '10,1,6',
 '15,2,6',
 '16,2,6',
 '17,1,6',
 '18,1,6',
 '20,1,6',
 '21,1,6',
 '22,1,6',
 '24,1,6',
 '25,5,6',
 '26,1,6',
 '28,1,6',
 '30,1,6',
 '33,1,6',
 '38,1,6',
 '39,1,6',
 '43,4,6',
 '44,1,6',
 '45,1,6',
 '48,1,6',
 '49,1,6',
 '50,1,6',
 '53,1,6',
 '54,1,6',
 '55,1,6',
 '56,1,6',
 '58,4,6',
 '59,1,6',
 '60,1,6',
 '61,3,6',
 '63,3,6',
 '66,1,6',
 '67,3,6',
 '68,1,6',
 '69,1,6',
 '71,2,6',
 '73,1,6',
 '75,1,6',
 '77,1,6',
 '79,1,6',
 '81,1,6',
 '84,1,6',
 '85,3,6',
 '86,1,6',
 '87,1,6',
 '88,1,6',
 '89,1,6',
 '91,2,6',
 '94,1,6',
 '95,2,6',
 '96,1,6',
 '1,1,7',
 '2,2,7',
 '3,1,7',
 '4,1,7',
 '7,1,7',
 '10,1,7',
 '11,2,7',
 '14,2,7',
 '15,1,7',
 '16,1,7',
 '18,1,7',
 '

In [0]:
movdf = spark.read \
  .format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", ',') \
  .option("path", "/FileStore/tables/movies.csv") \
  .load()

In [0]:
movdf.display()

movieId,rating,userId
2,3,0
3,1,0
5,2,0
9,4,0
11,1,0
12,2,0
15,1,0
17,1,0
19,1,0
21,1,0


In [0]:
usercntdf=movdf.groupBy('userID').count().sort(col('userID'))
usercntdf=usercntdf.withColumnRenamed('userID','userID_cnt')
usercntdf.display()

userID_cnt,count
0,49
1,49
2,46
3,48
4,55
5,49
6,57
7,54
8,49
9,53


The dataset has 3 columns, being the movieId, rating and userId. There are 1501 rows representing 1501 ratings by all users. 30 users and 100 movies are included.

The following analysis finds top 15 movies with the highest average ratings, top 15 movies with the highest number of ratings, top 10 users providing highest average ratings and top 10 users rated for the most times.

In [0]:
usersumdf=movdf.groupBy('userID').sum('rating').sort(col('userID'))
usersumdf=usersumdf.join(usercntdf,usersumdf.userID==usercntdf.userID_cnt).drop("userID_cnt").sort(col('userID'))
usersumdf.display()

userID,sum(rating),count
0,70,49
1,76,49
2,95,46
3,88,48
4,86,55
5,87,49
6,82,57
7,88,54
8,93,49
9,95,53


In [0]:
useravgdf=usersumdf.withColumn('User_avg', ( usersumdf['sum(rating)'] ) /( usersumdf['count'] )).sort(col('User_avg').desc())
useravgdf.show(10)
#10 users giving highest average ratings

+------+-----------+-----+------------------+
userID|sum(rating)|count| User_avg|
+------+-----------+-----+------------------+
 11| 128| 56|2.2857142857142856|
 26| 108| 49| 2.204081632653061|
 22| 121| 56|2.1607142857142856|
 23| 111| 52|2.1346153846153846|
 2| 95| 46|2.0652173913043477|
 17| 90| 46|1.9565217391304348|
 8| 93| 49|1.8979591836734695|
 24| 98| 52|1.8846153846153846|
 12| 102| 55|1.8545454545454545|
 3| 88| 48|1.8333333333333333|
+------+-----------+-----+------------------+
only showing top 10 rows

In [0]:
#10 users giving the largest number of ratings
useravgdf.sort(col('count').desc()).show(10)

+------+-----------+-----+------------------+
userID|sum(rating)|count| User_avg|
+------+-----------+-----+------------------+
 6| 82| 57|1.4385964912280702|
 14| 102| 57|1.7894736842105263|
 22| 121| 56|2.1607142857142856|
 11| 128| 56|2.2857142857142856|
 12| 102| 55|1.8545454545454545|
 4| 86| 55|1.5636363636363637|
 7| 88| 54|1.6296296296296295|
 9| 95| 53|1.7924528301886793|
 24| 98| 52|1.8846153846153846|
 23| 111| 52|2.1346153846153846|
+------+-----------+-----+------------------+
only showing top 10 rows

In [0]:
mvcntdf=movdf.groupBy('movieId').count().sort(col('movieId'))
mvcntdf=mvcntdf.withColumnRenamed('movieId','movieId_cnt')
mvcntdf.display()

movieId_cnt,count
0,16
1,13
2,19
3,13
4,17
5,13
6,20
7,16
8,7
9,16


In [0]:
mvsumdf=movdf.groupBy('movieId').sum('rating').sort(col('movieId'))
mvsumdf=mvsumdf.join(mvcntdf,mvsumdf.movieId==mvcntdf.movieId_cnt).drop("movieId_cnt").sort(col('movieId'))
mvsumdf.display()

movieId,sum(rating),count
0,20,16
1,18,13
2,40,19
3,17,13
4,30,17
5,20,13
6,29,20
7,30,16
8,13,7
9,23,16


In [0]:
mvavgdf=mvsumdf.withColumn('movie_avg', ( mvsumdf['sum(rating)'] ) /( mvsumdf['count'] ))
mvavgdf.sort(col('movie_avg').desc()).show(15)
#15 highest rated movies

+-------+-----------+-----+------------------+
movieId|sum(rating)|count| movie_avg|
+-------+-----------+-----+------------------+
 32| 35| 12|2.9166666666666665|
 90| 45| 16| 2.8125|
 30| 35| 14| 2.5|
 94| 47| 19| 2.473684210526316|
 23| 37| 15| 2.466666666666667|
 49| 39| 16| 2.4375|
 29| 48| 20| 2.4|
 18| 36| 15| 2.4|
 52| 33| 14| 2.357142857142857|
 62| 36| 16| 2.25|
 53| 27| 12| 2.25|
 92| 31| 14|2.2142857142857144|
 46| 22| 10| 2.2|
 68| 41| 19|2.1578947368421053|
 87| 32| 15|2.1333333333333333|
+-------+-----------+-----+------------------+
only showing top 15 rows

In [0]:
#15 mostly rated movies
mvavgdf.sort(col('count').desc()).show(15)

+-------+-----------+-----+------------------+
movieId|sum(rating)|count| movie_avg|
+-------+-----------+-----+------------------+
 6| 29| 20| 1.45|
 29| 48| 20| 2.4|
 51| 40| 20| 2.0|
 22| 41| 20| 2.05|
 50| 36| 20| 1.8|
 94| 47| 19| 2.473684210526316|
 55| 34| 19|1.7894736842105263|
 68| 41| 19|2.1578947368421053|
 2| 40| 19|2.1052631578947367|
 15| 22| 19|1.1578947368421053|
 85| 36| 18| 2.0|
 36| 33| 18|1.8333333333333333|
 86| 21| 18|1.1666666666666667|
 88| 37| 18|2.0555555555555554|
 45| 24| 18|1.3333333333333333|
+-------+-----------+-----+------------------+
only showing top 15 rows

In [0]:
mvavgdf.sort(col('movie_avg').asc()).show(10)

+-------+-----------+-----+------------------+
movieId|sum(rating)|count| movie_avg|
+-------+-----------+-----+------------------+
 78| 14| 14| 1.0|
 99| 14| 13|1.0769230769230769|
 15| 22| 19|1.1578947368421053|
 86| 21| 18|1.1666666666666667|
 84| 19| 16| 1.1875|
 0| 20| 16| 1.25|
 97| 19| 15|1.2666666666666666|
 61| 19| 15|1.2666666666666666|
 3| 17| 13|1.3076923076923077|
 45| 24| 18|1.3333333333333333|
+-------+-----------+-----+------------------+
only showing top 10 rows

Based on above analysis: 
Top 15 movies with the highest average ratings: 32,90,30,94,23,49,18,29,52,53,62,92,46,68,87, 
Top 15 movies with the highest number of ratings: 6,29,51,22,50,94,55,68,2,15,85,36,86,88,45, 
Top 10 users providing highest average ratings: 11,26,22,23,2,17,8,24,12,3, 
Top 10 users rated for the most times: 6,14,22,11,12,4,7,9,24,23

Next, the ALS based recommender is trained and tuned on hyperparameters.

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit

0.8:0.2 train-test ratio

In [0]:
(train, test) = movdf.randomSplit([0.8, 0.2])
train.show(5)

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 0| 1| 3|
 0| 1| 5|
 0| 1| 6|
 0| 1| 8|
 0| 1| 11|
+-------+------+------+
only showing top 5 rows

In [0]:
als = ALS(userCol= 'userId',itemCol= 'movieId',ratingCol='rating', coldStartStrategy = 'drop')
model=als.fit(train)

In [0]:
print('rank:',als._java_obj.getRank())
print('maxIter:',als._java_obj.getMaxIter())
print('regParam:',als._java_obj.getRegParam())
print('numItemBlocks:',als._java_obj.getNumItemBlocks())
print('numUserBlocks:',als._java_obj.getNumUserBlocks())


rank: 10
maxIter: 10
regParam: 0.1
numItemBlocks: 10
numUserBlocks: 10

In [0]:
predictions = model.transform(test)
mse_eval = RegressionEvaluator(metricName= 'mse', labelCol= 'rating',predictionCol= 'prediction')
rmse_eval = RegressionEvaluator(metricName= 'rmse', labelCol= 'rating',predictionCol= 'prediction')
mae_eval= RegressionEvaluator(metricName= 'mae', labelCol= 'rating',predictionCol= 'prediction')
mse = mse_eval.evaluate(predictions)
rmse = rmse_eval.evaluate(predictions)
mae = mae_eval.evaluate(predictions)
print("mse:",mse)
print("rmse:",rmse)
print("mae:",mae)
predictions.show()

mse: 0.958397665308887
rmse: 0.9789778676297473
mae: 0.6856936406028473
+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 11| 1| 0|0.89487255|
 12| 2| 0| 2.8282707|
 17| 1| 0| 0.8903369|
 19| 1| 0| 1.728325|
 26| 3| 0|0.59450287|
 27| 1| 0| 0.4784589|
 37| 1| 0| 0.5937365|
 51| 1| 0| 0.8981965|
 61| 2| 0| 1.0556384|
 64| 1| 0| 0.9383202|
 69| 1| 0|0.46846205|
 95| 2| 0| 1.4965026|
 3| 1| 1| 0.9681201|
 9| 3| 1| 1.2917466|
 12| 1| 1| 1.3744621|
 16| 1| 1| 0.9934349|
 40| 1| 1| 0.9252024|
 41| 2| 1| 1.871395|
 43| 1| 1| 1.2899225|
 73| 1| 1| 1.8923916|
+-------+------+------+----------+
only showing top 20 rows

0.7:0.3 train-test ratio

In [0]:
(train, test) = movdf.randomSplit([0.7, 0.3])
train.show(5)
als = ALS(userCol= 'userId',itemCol= 'movieId',ratingCol='rating', coldStartStrategy = 'drop')
model=als.fit(train)

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 0| 1| 5|
 0| 1| 8|
 0| 1| 11|
 0| 1| 15|
 0| 1| 20|
+-------+------+------+
only showing top 5 rows

In [0]:
als = ALS(userCol= 'userId',itemCol= 'movieId',ratingCol='rating', coldStartStrategy = 'drop')
model=als.fit(train)

In [0]:
predictions = model.transform(test)
mse_eval = RegressionEvaluator(metricName= 'mse', labelCol= 'rating',predictionCol= 'prediction')
rmse_eval = RegressionEvaluator(metricName= 'rmse', labelCol= 'rating',predictionCol= 'prediction')
mae_eval= RegressionEvaluator(metricName= 'mae', labelCol= 'rating',predictionCol= 'prediction')
mse = mse_eval.evaluate(predictions)
rmse = rmse_eval.evaluate(predictions)
mae = mae_eval.evaluate(predictions)
print("mse:",mse)
print("rmse:",rmse)
print("mae:",mae)
predictions.show()

mse: 1.2471419725286175
rmse: 1.1167551085751153
mae: 0.779371863253778
+-------+------+------+------------+
movieId|rating|userId| prediction|
+-------+------+------+------------+
 5| 2| 0| 1.1185006|
 12| 2| 0| 2.1652212|
 19| 1| 0| 0.76234543|
 29| 1| 0| 2.4267|
 47| 1| 0| 1.1489031|
 48| 1| 0|0.0020937324|
 64| 1| 0| 2.0875163|
 69| 1| 0| 0.8768036|
 72| 1| 0| 1.5119314|
 79| 1| 0| 0.55924207|
 83| 1| 0| 0.9006993|
 12| 1| 1| 1.4410698|
 21| 3| 1| 1.3636444|
 28| 3| 1| 1.1573578|
 57| 1| 1| 1.2907336|
 72| 1| 1| 1.2724011|
 77| 3| 1| 2.0414314|
 96| 1| 1| 1.2583891|
 12| 3| 2| 1.160589|
 26| 1| 2| 1.5092258|
+-------+------+------+------------+
only showing top 20 rows

Parameter tunning: Firstly manually decide some of the parameters (rank and regParams) and fix them, for saving computing effort for the further grid searches.

In [0]:
import numpy as np
(train, test) = movdf.randomSplit([0.8, 0.2])
ranks=[8,10,12]
maxIters=[20]
regParams=[0.1,1,10]
numItemBlockss=[10]
numUserBlockss=[10]
bestparams=[]
bestnum=0
bestmodel=None
besterr=99
bestpred=None
count=-1
for arank in ranks:
    for ami in maxIters:
        for arp in regParams:
            for anib in numItemBlockss:
                for anub in numUserBlockss:
                    count+=1
                    myals = ALS(userCol= 'userId',itemCol= 'movieId',ratingCol='rating', coldStartStrategy = 'drop',
                               rank=arank,maxIter=ami,regParam=arp,numItemBlocks=anib,numUserBlocks=anub)
                    mymodel=myals.fit(train)
                    mypredictions = mymodel.transform(test)
                    myeval = RegressionEvaluator(metricName= 'rmse', labelCol= 'rating',predictionCol= 'prediction')
                    err = myeval.evaluate(mypredictions)
                    if err<besterr:
                        besterr=err
                        bestmodel=mymodel.copy()
                        bestpred=mypredictions
                        bestnum=count  
                        bestparams=[arank,ami,arp,anib,anub]
                    print(count)
print("rmse:",besterr)
print("best parameters [rank,maxIter,regParam,numItemBlocks,numUserBlockss]:",bestparams)

0
1
2
3
4
5
6
7
8
rmse: 1.0809909067623016
best parameters [rank,maxIter,regParam,numItemBlocks,numUserBlockss]: [12, 20, 0.1, 10, 10]

Parameter tunning for remaining parameters fixing the tuned parameters from previous steps;
train-test ratio 8:2, rmse

In [0]:
(train, test) = movdf.randomSplit([0.8, 0.2])
als = ALS(userCol= 'userId',itemCol= 'movieId',ratingCol='rating', coldStartStrategy = 'drop')
parameters=ParamGridBuilder()\
    .addGrid(als.rank,[12])\
    .addGrid(als.maxIter, [20,40])\
    .addGrid(als.regParam, [0.1])\
    .addGrid(als.numItemBlocks, [10,20])\
    .addGrid(als.numUserBlocks, [10,20])\
.build()
eval = RegressionEvaluator(metricName= 'rmse', labelCol= 'rating',predictionCol= 'prediction')
trainvs = TrainValidationSplit(estimator=als,estimatorParamMaps=parameters, evaluator=eval)
cv = CrossValidator(estimator =als,estimatorParamMaps =parameters, evaluator =eval,numFolds=3)
model = trainvs.fit(train)

In [0]:
predictions = model.transform(test)
rmse = eval.evaluate(predictions)
print("rmse:",rmse)
predictions.show()

rmse: 0.9159836838569291
+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 4| 12| 0.8676408|
 31| 3| 14| 2.2171078|
 31| 2| 25| 1.5929376|
 31| 1| 26| 0.7479627|
 31| 1| 27| 0.6150324|
 31| 1| 29| 0.9358617|
 85| 1| 12|0.42660517|
 85| 2| 20| 2.0340848|
 53| 1| 7| 1.6153181|
 53| 5| 8| 2.5108628|
 53| 1| 12| 1.0948099|
 53| 5| 21| 2.341651|
 53| 1| 23| 1.2764945|
 78| 1| 8|0.88493913|
 78| 1| 12| 0.9326941|
 78| 1| 22| 1.3495789|
 34| 1| 4| 1.2881163|
 34| 1| 16| 1.3879718|
 81| 1| 7| 1.3465313|
 81| 1| 15| 1.616616|
+-------+------+------+----------+
only showing top 20 rows

Parameter tunning based on the roughly tuned parameters from previous steps;
train-test ratio 8:2, mae

In [0]:
(train, test) = movdf.randomSplit([0.8, 0.2])
als = ALS(userCol= 'userId',itemCol= 'movieId',ratingCol='rating', coldStartStrategy = 'drop')
parameters=ParamGridBuilder()\
    .addGrid(als.rank,[12])\
    .addGrid(als.maxIter, [20,40])\
    .addGrid(als.regParam, [0.1])\
    .addGrid(als.numItemBlocks, [10,20])\
    .addGrid(als.numUserBlocks, [10,20])\
.build()
eval = RegressionEvaluator(metricName= 'mae', labelCol= 'rating',predictionCol= 'prediction')
trainvs = TrainValidationSplit(estimator=als,estimatorParamMaps=parameters, evaluator=eval)
cv = CrossValidator(estimator =als,estimatorParamMaps =parameters, evaluator =eval,numFolds=3)
model2 = trainvs.fit(train)

In [0]:
predictions = model2.transform(test)
mae = eval.evaluate(predictions)
print("mae:",mae)
predictions.show()

mae: 0.6091065045176564
+-------+------+------+------------+
movieId|rating|userId| prediction|
+-------+------+------+------------+
 31| 1| 5| 1.4120657|
 31| 1| 26| 0.46876115|
 31| 1| 27| 0.93005663|
 85| 1| 5| 1.4485486|
 85| 1| 13| 1.3697956|
 85| 1| 15| 1.9944156|
 85| 2| 20| 1.878714|
 65| 1| 16| 0.61727273|
 65| 1| 22| 1.6965784|
 53| 3| 13| 2.835839|
 53| 3| 14| 3.3466127|
 78| 1| 1| 0.8569995|
 78| 1| 8| 0.89269084|
 78| 1| 22| 1.2236706|
 78| 1| 27| 0.83289135|
 34| 3| 3| 0.78095704|
 34| 1| 4| 0.94520354|
 34| 1| 16|-0.005653832|
 34| 1| 19| 0.19298117|
 81| 2| 5| 2.530323|
+-------+------+------+------------+
only showing top 20 rows

Parameter tunning based on the roughly tuned parameters from previous steps;
train-test ratio 7:3, rmse

In [0]:
(train, test) = movdf.randomSplit([0.7, 0.3])
als = ALS(userCol= 'userId',itemCol= 'movieId',ratingCol='rating', coldStartStrategy = 'drop')
parameters=ParamGridBuilder()\
    .addGrid(als.rank,[12])\
    .addGrid(als.maxIter, [20,40])\
    .addGrid(als.regParam, [0.1])\
    .addGrid(als.numItemBlocks, [10,20])\
    .addGrid(als.numUserBlocks, [10,20])\
.build()
eval = RegressionEvaluator(metricName= 'rmse', labelCol= 'rating',predictionCol= 'prediction')
trainvs = TrainValidationSplit(estimator=als,estimatorParamMaps=parameters, evaluator=eval)
cv = CrossValidator(estimator =als,estimatorParamMaps =parameters, evaluator =eval,numFolds=3)
model = trainvs.fit(train)

In [0]:
predictions = model.transform(test)
rmse = eval.evaluate(predictions)
print("rmse:",rmse)
predictions.show()

rmse: 1.023347938377496
+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 1| 5| 1.3737491|
 85| 3| 1| 1.5050097|
 85| 1| 2| 0.7374009|
 85| 4| 7| 2.3780792|
 85| 5| 8| 2.3222828|
 85| 1| 12| 1.7576412|
 85| 1| 15| 0.9025166|
 85| 5| 16| 1.5448868|
 65| 1| 2| 1.3166038|
 65| 2| 5| 2.6878073|
 65| 2| 15| 2.013025|
 65| 1| 24| 1.3546054|
 53| 1| 7| 1.9191986|
 53| 3| 13| 1.8100374|
 53| 3| 14| 3.992831|
 53| 1| 25| 1.1088554|
 78| 1| 8|0.97871697|
 78| 1| 13| 0.8028197|
 78| 1| 19|0.89869565|
 78| 1| 22| 1.1770544|
+-------+------+------+----------+
only showing top 20 rows

Parameter tunning based on the roughly tuned parameters from previous steps;
train-test ratio 7:3, mae

In [0]:
(train, test) = movdf.randomSplit([0.7, 0.3])
als = ALS(userCol= 'userId',itemCol= 'movieId',ratingCol='rating', coldStartStrategy = 'drop')
parameters=ParamGridBuilder()\
    .addGrid(als.rank,[12])\
    .addGrid(als.maxIter, [20,40])\
    .addGrid(als.regParam, [0.1])\
    .addGrid(als.numItemBlocks, [10,20])\
    .addGrid(als.numUserBlocks, [10,20])\
.build()
eval = RegressionEvaluator(metricName= 'mae', labelCol= 'rating',predictionCol= 'prediction')
trainvs = TrainValidationSplit(estimator=als,estimatorParamMaps=parameters, evaluator=eval)
cv = CrossValidator(estimator =als,estimatorParamMaps =parameters, evaluator =eval,numFolds=3)
model2 = trainvs.fit(train)

In [0]:
predictions = model2.transform(test)
mae = eval.evaluate(predictions)
print("mae:",mae)
predictions.show()

mae: 0.767892589362768
+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 1| 0| 0.7137946|
 31| 1| 4| 2.1567519|
 31| 1| 5| 1.1812205|
 31| 4| 12| 1.1865945|
 31| 1| 19|0.36266685|
 31| 1| 26|0.72980833|
 85| 1| 5|0.74584293|
 85| 4| 7| 1.1949837|
 85| 5| 8| 2.453227|
 85| 1| 13| 1.5723487|
 85| 5| 16| 1.3235414|
 65| 2| 3| 0.596998|
 65| 1| 4| 0.744275|
 65| 1| 16| 0.8936615|
 65| 1| 22| 1.0298033|
 65| 5| 23| 2.2105517|
 65| 1| 24| 0.5863366|
 53| 1| 6| 2.448483|
 53| 1| 9| 1.409443|
 53| 3| 20| 1.7812586|
+-------+------+------+----------+
only showing top 20 rows

Finally, we test out the best model from last part on users 9 and 13.

In [0]:
def missedInts(left,right,existInts):
    allints=list(range(left,right+1))
    retints=[]
    for aint in allints:
        if not aint in existInts:
            retints.append(aint)
    return retints
missedInts(0,10,[1,2,5,9])

Out[51]: [0, 3, 4, 6, 7, 8, 10]

In [0]:
import numpy as np
u9rated=movdf.where(movdf['userId']==9)
u9ratedmvs=u9rated.select('movieId').rdd.flatMap(lambda x: x).collect()
u9missedmvs=missedInts(0,99,u9ratedmvs)
nummissed=len(u9missedmvs)
missed2d=np.zeros([nummissed,2])
missed2d[:,0]=u9missedmvs
missed2d[:,1]=np.ones(nummissed)*9
missed2d=missed2d.astype(int)
columns=['movieId','userId']
u9misseddf= spark.createDataFrame(missed2d.tolist(), columns)
u9misseddf.show()

+-------+------+
movieId|userId|
+-------+------+
 0| 9|
 1| 9|
 8| 9|
 10| 9|
 11| 9|
 13| 9|
 16| 9|
 17| 9|
 18| 9|
 20| 9|
 23| 9|
 27| 9|
 28| 9|
 29| 9|
 31| 9|
 33| 9|
 34| 9|
 40| 9|
 41| 9|
 42| 9|
+-------+------+
only showing top 20 rows

In [0]:
u9preds = model2.transform(u9misseddf)
u9preds=u9preds.sort(col('prediction').desc())
u9preds.display(10)

movieId,userId,prediction
62,9,3.0348744
46,9,2.968433
17,9,2.9584653
23,9,2.9104004
13,9,2.8551896
29,9,2.8470316
27,9,2.6728992
55,9,2.645823
65,9,2.6343143
48,9,2.56945


In [0]:
u9rec10=u9preds.select('movieId').rdd.flatMap(lambda x: x).collect()
print("top 10 movies recommended to user 9:")
u9rec10[0:10]

top 10 movies recommended to user 9:
Out[65]: [62, 46, 17, 23, 13, 29, 27, 55, 65, 48]

In [0]:
predictions = model.transform(test)
rmse = eval.evaluate(predictions)
print("rmse:",rmse)
predictions.show()

rmse: 0.9819464566729535
+-------+------+------+-----------+
movieId|rating|userId| prediction|
+-------+------+------+-----------+
 31| 1| 0| 1.587599|
 31| 1| 4| 1.9313468|
 31| 3| 8| 2.415783|
 31| 4| 12| 1.2760637|
 31| 1| 18| 1.5813684|
 31| 1| 27| 1.1051265|
 85| 1| 4| 1.981026|
 85| 3| 6| 2.1473768|
 85| 5| 8| 2.257608|
 85| 1| 15| 1.0353209|
 85| 3| 21| 1.6297748|
 85| 1| 28| 0.25987715|
 65| 2| 3| 1.3484352|
 65| 1| 4| 1.2945793|
 53| 5| 8| 2.9149005|
 53| 1| 12|-0.30048117|
 53| 1| 23| 0.11456144|
 53| 1| 25|-0.19856736|
 78| 1| 11| 0.8846265|
 78| 1| 13| 0.7564558|
+-------+------+------+-----------+
only showing top 20 rows

In [0]:
u13rated=movdf.where(movdf['userId']==13)
u13ratedmvs=u13rated.select('movieId').rdd.flatMap(lambda x: x).collect()
u13missedmvs=missedInts(0,99,u13ratedmvs)
nummissed=len(u13missedmvs)
missed2d=np.zeros([nummissed,2])
missed2d[:,0]=u13missedmvs
missed2d[:,1]=np.ones(nummissed)*13
missed2d=missed2d.astype(int)
columns=['movieId','userId']
u13misseddf= spark.createDataFrame(missed2d.tolist(), columns)
u13misseddf.show()

+-------+------+
movieId|userId|
+-------+------+
 1| 13|
 2| 13|
 7| 13|
 8| 13|
 9| 13|
 10| 13|
 11| 13|
 13| 13|
 16| 13|
 19| 13|
 23| 13|
 24| 13|
 25| 13|
 28| 13|
 30| 13|
 32| 13|
 34| 13|
 35| 13|
 36| 13|
 37| 13|
+-------+------+
only showing top 20 rows

In [0]:
u13preds = model2.transform(u13misseddf)
u13preds=u13preds.sort(col('prediction').desc())
u13preds.display()

movieId,userId,prediction
30,13,2.3758736
2,13,2.3649511
41,13,2.269994
70,13,2.2300198
69,13,2.1825686
32,13,2.1633108
75,13,2.1183453
92,13,1.9785659
76,13,1.9566662
8,13,1.8824189


In [0]:
u13rec10=u13preds.select('movieId').rdd.flatMap(lambda x: x).collect()
print("top 10 movies recommended to user 13:")
u13rec10[0:10]

top 10 movies recommended to user 13:
Out[68]: [30, 2, 41, 70, 69, 32, 75, 92, 76, 8]